In [ ]:
%matplotlib inline

# 第2部 特集4 Python で画像認識にチャレンジ (1)

----

## 第1章

- 画像認識の代表的な問題

    - 移動体検知
    - 物体認識
    - 物体検出
    - 類似画像検索


----

## 第2章 環境構築

 $ pip install scikit-learn scikit-image matplotlib

### scikit-image

◯リスト1 画像の読み書き・表示

In [ ]:
from skimage import io

In [ ]:
image = io.imread('mandrill.png') #画像ファイルの読み込み
io.imshow(image)                  #画像ファイルの表示
#io.show()                        # jupyter では不要?

In [ ]:
%rm mandrill2.png
%ls -la mandrill2.png

In [ ]:
io.imsave('mandrill2.png',image) #画像ファイルの保存

In [ ]:
%ls -la mandrill2.png

◯リスト2 画像の表現を取得

In [ ]:
image = io.imread('mandrill.png')

In [ ]:
print '(1)', type(image)     # (1) imageの型名

In [ ]:
print '(2)', image.shape     # (2) image 各次元の大きさ

In [ ]:
print '(3)', image[300,400]  # (3) 座標(300,400)の画素値

◯リスト3 画像の要素に対する操作

In [ ]:
image[240,240, 0:3] = [0, 0, 255] #中央ピクセルを青くする
io.imshow(image)                  #画像ファイルの表示

In [ ]:
image[20:140, 20:200, 0:3] = [0,0,0]  # (20,20)から(140,200)までの四角を黒に
#      Y軸      X軸    要素
io.imshow(image)                      #画像ファイルの表示

In [ ]:
image[300:400, 100:400, 0:3] *= 0.5  # (300,100)から(400,400)までの四角を薄くする
#      Y軸      X軸    要素
io.imshow(image)                      #画像ファイルの表示

----

次の章の準備で、マンドリルの目を切り出しておく

In [ ]:
image = io.imread('mandrill.png') #画像ファイルの読み込み
io.imshow(image)                  #画像ファイルの表示
#io.show()                        # jupyter では不要?

In [ ]:
io.imsave('eye.png',image[40:80,145:185]) #目玉部分を切り出して eye.png ファイルに保存
io.imshow('eye.png')                      #eye.png を表示

----

## 第3章 シンプルな画像認識を実装

◯リスト1 単純なテンプレートマッチング

In [ ]:
import sys
import numpy as np
from skimage  import io
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# テンプレートファイルと評価画像の指定
template_path = 'eye.png'
target_path   = 'mandrill.png'

In [ ]:
# ファイルをグレースケール(Gray1次元の色表現)で取得
template = io.imread(template_path, as_grey=True)
target   = io.imread(target_path,   as_grey=True)

In [ ]:
io.imshow(template)

In [ ]:
template

In [ ]:
th, tw   = template.shape # テンプレート画像のサイズを取得
th , tw , template.shape , target.shape

In [ ]:
# 類似度を図るために、差の2乗和(Sum of Squard Difference) を格納する 440 x 440 の 0 埋め配列を生成
score_map = np.zeros( shape = (target.shape[0] - th,
                               target.shape[1] - tw))
score_map.shape

In [ ]:
# 画像全体を1pixcelづつ移動して SSD を計算
for y in range(score_map.shape[0]): # Y軸
    for x in range(score_map.shape[1]): # X軸
        diff = target[y : y+th, x : x+tw] - template #ターゲットより 40x40の範囲を切り出して、Grayの差をとる
        score_map[y,x] = np.square(diff).sum()       #40x40点のGray差の二乗和をとる  
        if x == 1 and y == 1:
            print "diff is ",diff
            print "SSD is ",score_map[x,y]

In [ ]:
# SSDが最小の座標を取得
x, y = np.unravel_index(np.argmin(score_map),score_map.shape)
x, y

In [ ]:
# 結果を可視化
fig, (ax1,ax2,ax3) = plt.subplots(ncols=3, figsize=(8,3))

ax1.imshow(template, cmap=cm.Greys_r)
ax1.set_axis_off()
ax1.set_title('template')

ax2.imshow(target, cmap=cm.Greys_r)
ax2.set_axis_off()
ax2.set_title('target')
ax2.add_patch(plt.Rectangle((y,x), tw, th, edgecolor='w', facecolor='none', linewidth=2.5)) #マッチ位置


ax3.imshow(score_map, cmap=cm.Greys_r)
ax3.set_axis_off()
ax3.set_title('score_map')
ax3.add_patch(plt.Rectangle((y - th / 2,x - tw/2), tw, th, edgecolor='w', facecolor='none', linewidth=2.5)) #マッチ位置

----

◯リスト2 拡大縮小に対応したテンプレートマッチング

In [ ]:
# 小さなテンプレート画像を作る
from skimage.transform import rescale
image = io.imread('eye.png', as_grey=True)
smallImage = rescale(image, 0.3) # スケールを変更する
io.imshow(smallImage)
smallImage.shape

In [ ]:
template = smallImage                              # テンプレートのスケールが小さくなった
target = io.imread(target_path,   as_grey=True)    # 今までと同じターゲット

In [ ]:
def compute_score_map(template, target):
    th, tw = template.shape
    score_map = np.zeros( shape = (target.shape[0] - th,
                                   target.shape[1] - tw))
    # 画像全体を1pixcelづつ移動して SSD を計算
    for y in range(score_map.shape[0]): # Y軸
        for x in range(score_map.shape[1]): # X軸
            diff = target[y : y+th, x : x+tw] - template #ターゲットより 40x40の範囲を切り出して、Grayの差をとる
            score_map[y,x] = np.square(diff).sum()       #40x40点のGray差の二乗和をとる  # score_mapの座標が y,x に注目!
    print "min score is " , np.min(score_map)
    return score_map

In [ ]:
from skimage.transform import rescale

score_maps = []
scale_factor = 2.0 ** (-1.0/8.0)
target_scaled = target + 0
while target_scaled.shape > template.shape: #サンプルでは 8回決め打ちだったけど、テンプレートより小さくなるまでやる様変更
#for s in range(8):
    print "target shape is ", target_scaled.shape
    score_maps.append( compute_score_map(template, target_scaled))
    target_scaled = rescale(target_scaled, scale_factor)


In [ ]:
score, s, (x,y) = min([(np.min(score_map), s,
                        np.unravel_index(np.argmin(score_map), score_map.shape))
                      for s, score_map in enumerate(score_maps)])

In [ ]:
# 結果を可視化
fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(8,3))

ax1.imshow(template, cmap=cm.Greys_r)
ax1.set_axis_off()
ax1.set_title('template')

ax2.imshow(target, cmap=cm.Greys_r)
ax2.set_axis_off()
ax2.set_title('target')
scale = (scale_factor ** s)
th, tw = template.shape
rect = plt.Rectangle((y / scale, x / scale), tw / scale, th / scale, edgecolor='r', facecolor='none') #マッチ位置
ax2.add_patch(rect)
